# Local Embeddings

We will try a few embedding models running locally and compare their performance

## References

- https://docs.llamaindex.ai/en/stable/examples/embeddings/huggingface.html#huggingfaceembedding
- Leaderboard : https://huggingface.co/spaces/mteb/leaderboard
- Explaining leaderboard: https://huggingface.co/blog/mteb

## Basic Setup

### GPU

GPUs can really accellerate LLM / embedding operations.  Let's make sure we are using GPUs if we have them

In [9]:
import os
import torch

## To disable GPU and experiment, uncomment the following line
## Normally, you would want to use GPU, if one is available.
# os.environ["CUDA_VISIBLE_DEVICES"]=""

print ("using CUDA/GPU: ", torch.cuda.is_available())

for i in range(torch.cuda.device_count()):
   print("device ", i , torch.cuda.get_device_properties(i).name)

using CUDA/GPU:  True
device  0 NVIDIA GeForce RTX 2070


### Logging

To see every thing set the logging level to DEBUG.  
To disable all logs set the logging level to ERROR or WARNING

In [2]:
## Setup logging

import sys
import logging

# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Option 1 - Using Llama-Index

In [3]:
import os
## Optional, but set llamaindex cache dir to ./cache dir here.  Default is system tmp
## This way, we can easily see downloaded artifacts
os.environ['LLAMA_INDEX_CACHE_DIR'] = os.path.join(os.path.abspath(''), 'cache')

from llama_index.embeddings import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
embeddings = embed_model.get_text_embedding("Hello World!")

print(len(embeddings))
print(embeddings[:5])

/home/sujee/anaconda3/envs/atlas-2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


384
[-0.0032757227309048176, -0.011690807528793812, 0.041559189558029175, -0.03814816102385521, 0.024183066561818123]


### Let's try a few embedding models

See hugging face embedding models (sentence transformers) here : https://huggingface.co/models?library=sentence-transformers&sort=trending

Here are a select models for comparison.  Taken from leaderboard : https://huggingface.co/spaces/mteb/leaderboard

| model name                              | overall score | model size | model params | embedding length | License  | url                                                            |
|-----------------------------------------|---------------|------------|--------------|------------------|----------|----------------------------------------------------------------|
| intfloat/e5-mistral-7b-instruct         | 66.x          | 15 GB      | 7.11 B       | 4096             | MIT      | https://huggingface.co/intfloat/e5-mistral-7b-instruct         |
| BAAI/bge-large-en-v1.5                  | 64.x          | 1.34 GB    | 335 M        | 1024             | MIT      | https://huggingface.co/BAAI/bge-large-en-v1.5                  |
| BAAI/bge-small-en-v1.5                  | 62.x          | 133 MB     | 33.5 M       | 384              | MIT      | https://huggingface.co/BAAI/bge-small-en-v1.5                  |
| sentence-transformers/all-mpnet-base-v2 | 57.8          | 438 MB     |              | 768              | Apache 2 | https://huggingface.co/sentence-transformers/all-mpnet-base-v2 |
| sentence-transformers/all-MiniLM-L12-v2 | 56.x          | 134 MB     |              | 384              | Apache 2 | https://huggingface.co/sentence-transformers/all-MiniLM-L12-v2 |
| sentence-transformers/all-MiniLM-L6-v2  | 56.x          | 91 MB      |              | 384              | Apache 2 | https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2  |

### Benchmark

Benchmarks are fun way to evalute the performance of embedding models.

Here we use Python's `%timeit` command.  it runs the code a few times and calcualtes the average.

The following code block will calculate benchmark numbers.

Here are the benchmark numbers for encoding "Hello World" on my desktop 
- Ubuntu 22.04 with Nvidia CUDA drivers (Driver Version: 525.147.05   CUDA Version: 12.0)
- Intel 8 core CPU @ 3.6 GHZ
- 32 GB Memory
- Nvidia GEFORCE 2070 with 8 GB memory

As you can see, larger models tend take longer to execute.

You can also see GPU can really accelerate execution time!

At the end, it is a trade off between **accuracy and performancee**

| Model                                   | model size | embedding length | Execution time in ms (with GPU) | Execution time in ms (without GPU) |
|-----------------------------------------|------------|------------------|---------------------------------|------------------------------------|
| BAAI/bge-large-en-v1.5                  | 1.34 GB    | 1024             | 12.5                            | 67.7                               |
| BAAI/bge-small-en-v1.5                  | 133 MB     | 384              | 6.5                             | 9.4                                |
| sentence-transformers/all-mpnet-base-v2 | 438 MB     | 768              | 6.65                            | 20.5                               |
| sentence-transformers/all-MiniLM-L12-v2 | 134 MB     | 384              | 6.74                            | 9.4                                |
| sentence-transformers/all-MiniLM-L6-v2  | 91 MB      | 384              | 3.68                            | 4.8                                |

In [4]:
embedding_models = [
    'BAAI/bge-large-en-v1.5' ,
    'BAAI/bge-small-en-v1.5' ,
    'sentence-transformers/all-mpnet-base-v2' ,
    'sentence-transformers/all-MiniLM-L12-v2' ,
    'sentence-transformers/all-MiniLM-L6-v2' ,
]

import time
import timeit

for model in embedding_models:
    embed_model = HuggingFaceEmbedding(model_name=model)

    embeddings = embed_model.get_text_embedding("Hello World!")
    print(f'model={model}, embeding_length={len(embeddings):,}')
    %timeit (embed_model.get_text_embedding("Hello World!"))
    print()


model=BAAI/bge-large-en-v1.5, embeding_length=1,024
12.6 ms ± 143 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

model=BAAI/bge-small-en-v1.5, embeding_length=384
6.66 ms ± 179 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

model=sentence-transformers/all-mpnet-base-v2, embeding_length=768
6.81 ms ± 107 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

model=sentence-transformers/all-MiniLM-L12-v2, embeding_length=384
6.78 ms ± 149 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

model=sentence-transformers/all-MiniLM-L6-v2, embeding_length=384
3.67 ms ± 157 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



## Option 2: Encoding Using Sentence Transformers

In [5]:
from sentence_transformers import SentenceTransformer
#sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
# embeddings = model.encode(sentences)
embeddings = model.encode('a happy dog!')
print(model)
print (len(embeddings))
print(embeddings[:5])

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda
Use pytorch device: cuda


Batches: 100%|██████████| 1/1 [00:00<00:00, 269.80it/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)
384
[-0.00542399  0.07206922 -0.02727443  0.04371347 -0.0695779 ]
